In [1]:

import pandas as pd
from transformers import AutoTokenizer, AutoModel
from datetime import datetime
from datetime import datetime
from zipfile import ZipFile
import logging
import math
from torch.utils.data import DataLoader, Subset
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import torch
from torch.optim import AdamW
from sklearn.model_selection import KFold



/users/sgdbareh/volatile/miniconda3/envs/ECHR/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_batch_size = 16
train = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/BERT-rerank/BERT_training_data_df.pkl')

label2int = {"neg": 0, "pos": 1}

train_samples = []

# Iterate over the DataFrame rows
for index, row in train.iterrows():
    # Create InputExample for positive label
    train_samples.append(InputExample(texts=[row['Comm_Case'], row['Positive']], label=label2int["pos"]))
    # Create InputExample for negative label
    train_samples.append(InputExample(texts=[row['Comm_Case'], row['Negative']], label=label2int["neg"]))



# Create subset dataloaders for the current fold
#train_subset = Subset(train_samples, train_index)
#val_subset = Subset(train_samples, val_index)

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
#val_dataloader = DataLoader(val_subset, shuffle=False, batch_size=train_batch_size)



In [9]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size, collate_fn=util.batch_to_device)


In [11]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size, collate_fn=lambda batch: util.batch_to_device(batch, 'cuda'))


In [12]:
# Iterate over the DataLoader and print the first batch
for batch in train_dataloader:
    for example in batch:
        print(f"Texts: {example.texts}, Label: {example.label}")
    break  

TypeError: list indices must be integers or slices, not InputExample